In [ ]:
# --- 1) Mount Google Drive ---
from google.colab import drive
drive.mount('/content/drive')

# --- 2) Point to model folder in Drive ---
MODEL_DIR = "/content/drive/MyDrive/model"

# --- 3) Load tokenizer + model from Drive ---
!pip install -q transformers torch gradio

import os, torch, torch.nn.functional as F
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import gradio as gr

assert os.path.isdir(MODEL_DIR), f"Model dir not found: {MODEL_DIR}. Check the path in Drive."

tokenizer = AutoTokenizer.from_pretrained(MODEL_DIR)
model = AutoModelForSequenceClassification.from_pretrained(MODEL_DIR)
model.eval()

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

labels = ["Legit", "Fraud"]  

@torch.no_grad()
def predict(text):
    enc = tokenizer(text, truncation=True, padding=True, max_length=256, return_tensors="pt").to(device)
    outputs = model(**enc)
    probs = F.softmax(outputs.logits, dim=-1).squeeze().detach().cpu().tolist()
    pred_idx = int(torch.tensor(probs).argmax())
    pred = labels[pred_idx]
    return pred, {labels[i]: float(probs[i]) for i in range(len(labels))}

demo = gr.Interface(
    fn=predict,
    inputs="text",
    outputs=["label", "json"],
    title="Fake Job Detector",
    description="Paste a job posting to check if it's Legit or Fraud"
)

demo.launch(share=True)


Mounted at /content/drive
Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://a8c5bd7993c939a57b.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
